<a href="https://colab.research.google.com/github/Shreyas-ctrl/Certifications/blob/main/Medical_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install Dependencies
!pip install -qU google-generativeai==0.8.5 google-ai-generativelanguage==0.6.15 langgraph langchain langchain-google-genai openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 24.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.45.0 which is incompatible.


In [ ]:
#Import and secure API key input
import os
import getpass#Hide API keys
from langgraph.graph import StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage

In [ ]:
#Secure Gemini API Key Input
os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter your api key")

Enter your api key··········


In [ ]:
#Initialize Gemini 2.5 Flash
llm=ChatGoogleGenerativeAI(model="models/gemini-2.5-flash",temperature=0.2)

In [ ]:
#Node to ask user for symptom
def get_symptom(state: dict) -> dict:
  symptom=input("Welcome to XYZ Hospital,Please enter your symptom")
  state["symptom"]=symptom
  return state


In [ ]:
#Node to classify the symmptom
def classify_symptom(state:dict)->dict:
  prompt=(
      "you are a helpful medical assistant,classify the input into one of the categories"
      "general,Emergency,mental health"
      f"symptom : {state['symptom']}"
      'respond only with one word:general,emergency or mental'
      "#Example: input:I have fever,output :general"
  )

  response=llm.invoke([HumanMessage(content=prompt)])
  category=response.content.strip()
  print(f"LLM Classifies the symptom as:{category}")
  state["category"]=category
  return state

In [ ]:
#Router logic to route to correct node
def symptom_router(state:dict)->dict:
  cat=state["category"].lower()
  if "general" in cat:
    return "general"
  if "emergency" in cat:
    return "emergency"
  if "mental" in cat:
    return "mental"
  if "general" in cat:
    return "general"


In [ ]:
#Category-specific response nodes
def general_node(state: dict) -> dict:
  state["answer"] = f" '{state['symptom']}' : seems general : directing you to general ward for consulting a doctor"
  return state

def emergency_node(state:dict) -> dict:
  state["answer"] =f" ' {state['symptom']}': It is a Medical Emergency : seeking immediate help"
  return state

def mental_health_node(state: dict) -> dict:
  state["answer"] = f" '{state['symptom']}: seems like a medical health issue: talk to our counsellor"
  return state

In [ ]:
#Build a Langgraph
builder = StateGraph(dict)

#define the nodes
builder.set_entry_point("get_symptom")
builder.add_node("get_symptom", get_symptom)
builder.add_node("classify", classify_symptom)
builder.add_node("general", general_node)
builder.add_node("emergency", emergency_node)
builder.add_node("mental_health", mental_health_node)

builder.add_edge("get_symptom", "classify")
builder.add_conditional_edges("classify", symptom_router, {
    "general": "general",
    "emergency": "emergency",
    "mental_health": "mental_health"
})

builder.add_edge("general", END)
builder.add_edge("emergency", END)
builder.add_edge("mental_health", END)

In [ ]:
#Compile and invoke the graph
graph = builder.compile()
final_state = graph.invoke({})
print("final Output \n")
print(final_state["answer"])

Welcome to XYZ Hospital,Please enter your symptomI have mild cough and cold along with headache and joint pain.
LLM Classifies the symptom as:general
final Output 

 'I have mild cough and cold along with headache and joint pain.' : seems general : directing you to general ward for consulting a doctor
